# Test jypyter Notebook

In [1]:
!hostname

bfc10bb310f0


In [2]:
!which python

In [ ]:
!pip list installed

In [3]:
print("Hello Jupiter")

Hello Jupiter


In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
# we are using v1 api for Spark, and v2 for Dremio. 
# Eventually we will use v2 for both. 
MINIO_ACCESS_KEY = "K3zor6ZAUwbBrfiGPGwJ"
MINIO_SECRET_KEY = "46WaMwDWdaHEHin1KECq2FTAGllxETtyfGDMi9HI"

# How to configure Iceberg for Spark
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie3', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie3.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie3.ref', 'main')
        .set('spark.sql.catalog.nessie3.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie3.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie3.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie3.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie3.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5122c803-0438-4b3f-826b-7111e55f7473;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

23/12/06 16:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


In [ ]:
print(numpy.__version__)

In [5]:
## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023_4")

In [6]:
spark.sql("SELECT Count(*) as Total FROM csv_open_2023_4").show()

+------+
| Total|
+------+
|302231|
+------+



In [7]:
## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie3.df_open_2023_lesson4 USING iceberg AS SELECT * FROM csv_open_2023_4").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".    (0 + 5) / 5]
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++



In [8]:
## QUERY THE ICEBERG TABLE
spark.sql("SELECT * FROM nessie3.df_open_2023_lesson4 limit 10").show()
## GROUP BY QUERY
spark.sql("SELECT countryOfOriginCode,count(*) FROM nessie3.df_open_2023_lesson4 GROUP BY countryOfOriginCode").show()

+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|
|      300638|   Tola Morakinyo|     Tola|  Morakinyo|   ACT|     M|                 US|    

+-------------------+--------+
|countryOfOriginCode|count(1)|
+-------------------+--------+
|                 CA|   12862|
|                 US|  121594|
|                 MX|    3228|
|                 AU|   15560|
|                 GB|   20931|
|                 ES|    9403|
|                 GR|     309|
|                 ZA|    5520|
|                 DK|    1880|
|                 FR|   22228|
|                 LV|      99|
|                 RS|     103|
|                 IS|     930|
|                 AR|     532|
|                 PL|     619|
|                 IT|    6805|
|                 EE|     146|
|                 IE|    3962|
|                 NZ|    2992|
|                 IR|     619|
+-------------------+--------+
only showing top 20 rows



In [11]:
# ## CREATE A BRANCH WITH NESSIE
spark.sql("CREATE BRANCH IF NOT EXISTS lesson4 IN nessie3")

DataFrame[refType: string, name: string, hash: string]

In [12]:
# ## SWTICH TO THE NEW BRANCH
spark.sql("USE REFERENCE lesson4 IN nessie3")

DataFrame[refType: string, name: string, hash: string]

In [13]:
# ## DELETE ALL RECORDS WHERE countryOfOriginCode = 'FR'
spark.sql("DELETE FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode = 'FR'")

DataFrame[]

In [14]:
# 
spark.sql("SELECT count(*) FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode='FR'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [15]:
# ## SWITCH BACK TO MAIN BRANCH
spark.sql("USE REFERENCE main IN nessie3")

DataFrame[refType: string, name: string, hash: string]

In [16]:
# 
spark.sql("SELECT count(*) FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode='FR'").show()

+--------+
|count(1)|
+--------+
|   22228|
+--------+



In [17]:
# ## MERGE THE CHANGES
spark.sql("MERGE BRANCH lesson4 INTO main IN nessie3")

DataFrame[name: string, hash: string]

In [18]:
# 
spark.sql("SELECT count(*) FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode='FR'").show()

+--------+
|count(1)|
+--------+
|   22228|
+--------+



In [19]:
# ## SWITCH BACK TO MAIN BRANCH
spark.sql("USE REFERENCE main IN nessie3")
# 
spark.sql("SELECT count(*) FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode='FR'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [20]:
# ## SWITCH BACK TO MAIN BRANCH
spark.sql("USE REFERENCE lesson4 IN nessie3")
spark.sql("SELECT count(*) FROM nessie3.df_open_2023_lesson4 WHERE countryOfOriginCode='FR'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [22]:
spark.sql("SELECT countryOfOriginCode, count(*) FROM nessie3.df_open_2023_lesson4 GROUP BY countryOfOriginCode ORDER BY 2 DESC").show()

+-------------------+--------+
|countryOfOriginCode|count(1)|
+-------------------+--------+
|                 US|  121594|
|                 GB|   20931|
|                 AU|   15560|
|                 BR|   12996|
|                 CA|   12862|
|                 ES|    9403|
|                 KR|    6832|
|                 IT|    6805|
|                 DE|    6713|
|                 ZA|    5520|
|                 NL|    5492|
|                 IE|    3962|
|                 SE|    3686|
|                 CH|    3386|
|                 MX|    3228|
|                 NZ|    2992|
|                 NO|    2711|
|                 PT|    2012|
|                 BE|    1997|
|                 FI|    1896|
+-------------------+--------+
only showing top 20 rows



In [4]:
# 
spark.sql("DESCRIBE TABLE nessie3.df_open_2023_lesson4").show()

AnalysisException: Table or view not found: nessie3.df_open_2023_lesson4; line 1 pos 15;
'DescribeRelation false, [col_name#6, data_type#7, comment#8]
+- 'UnresolvedTableOrView [nessie3, df_open_2023_lesson4], DESCRIBE TABLE, true


In [5]:
spark.sql("SELECT uuid()").show()

+--------------------+
|              uuid()|
+--------------------+
|2415e707-e5ca-49e...|
+--------------------+



In [6]:
spark.sql("SELECT uuid() AS table_id,\"First row\" AS value").show()

+--------------------+---------+
|            table_id|    value|
+--------------------+---------+
|86e205e1-fbaf-42b...|First row|
+--------------------+---------+



In [7]:
spark.sql("""
CREATE TABLE nessie3.wap_table
    using iceberg
    tblproperties ('write.parquet.compression-codec' = 'zstd',
                   'write.wap.enabled'='true'
    )
      AS
        SELECT uuid() AS table_id,
               "\First row\" AS value
""").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".    (0 + 1) / 1]
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++



In [8]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [13]:
spark.sql("SELECT * FROM nessie3.wap_table").show()

spark.sql("ALTER TABLE nessie3.wap_table CREATE branch wap_branch").show()
spark.sql("ALTER TABLE nessie3.wap_table SET TBLPROPERTIES ('write.wap.enabled'='true')").show()
spark.sql("SET spark.wap.branch = wap_branch").show()
spark.sql("ALTER TABLE nessie3.wap_table ADD COLUMN nop string").show()


+--------------------+---------+
|            table_id|    value|
+--------------------+---------+
|fa635369-2df0-4bf...|First row|
+--------------------+---------+

23/12/06 04:47:54 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
++
||
++
++

++
||
++
++

+----------------+----------+
|             key|     value|
+----------------+----------+
|spark.wap.branch|wap_branch|
+----------------+----------+

23/12/06 04:47:56 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up
++
||
++
++



In [16]:
spark.conf.set('spark.wap.branch', 'wap_branch')
spark.sql("DESCRIBE nessie3.wap_table.branch_wap_branch").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|       table_id|   string|       |
|          value|   string|       |
|               |         |       |
| # Partitioning|         |       |
|Not partitioned|         |       |
+---------------+---------+-------+



In [17]:
spark.conf.set('spark.wap.branch', 'main')
spark.sql("DESCRIBE nessie3.wap_table.branch_main").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|       table_id|   string|       |
|          value|   string|       |
|            nop|   string|       |
|               |         |       |
| # Partitioning|         |       |
|Not partitioned|         |       |
+---------------+---------+-------+



In [19]:
spark.sql("insert into nessie3.wap_table.branch_wap_branch values ('11212', 'second val')").show()


Py4JJavaError: An error occurred while calling o51.sql.
: org.apache.iceberg.exceptions.ValidationException: Cannot write to both branch and WAP branch, but got branch [wap_branch] and WAP branch [main]
	at org.apache.iceberg.exceptions.ValidationException.check(ValidationException.java:49)
	at org.apache.iceberg.spark.SparkWriteConf.branch(SparkWriteConf.java:364)
	at org.apache.iceberg.spark.source.SparkWrite.<init>(SparkWrite.java:124)
	at org.apache.iceberg.spark.source.SparkWriteBuilder$1.<init>(SparkWriteBuilder.java:165)
	at org.apache.iceberg.spark.source.SparkWriteBuilder.build(SparkWriteBuilder.java:164)
	at org.apache.spark.sql.execution.datasources.v2.ExtendedV2Writes$$anonfun$apply$1.applyOrElse(ExtendedV2Writes.scala:58)
	at org.apache.spark.sql.execution.datasources.v2.ExtendedV2Writes$$anonfun$apply$1.applyOrElse(ExtendedV2Writes.scala:55)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$transformDownWithPruning$1(TreeNode.scala:584)
	at org.apache.spark.sql.ca

In [8]:
spark.sql("SHOW DATABASES").show()
spark.sql("SHOW NAMESPACES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+
|namespace|
+---------+
|  default|
+---------+



In [5]:
!which spark-sql